In [1]:
import nltk
import pandas as pd
import numpy as np
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Load Data
Load "izquierda diario" collected data from csv.

In [2]:
df = pd.read_csv('./data/izq_econ_news.csv')

In [3]:
df.shape

(1032, 7)

In [4]:
df.head(5)

,Unnamed: 0,date,flyer,lead,headline,body,url
0,0,\nMartes 22 de diciembre,LUCRO CAPITALISTA,Varios barrios fueron afectados por la falta d...,Masivo corte de luz: Edesur y los servicios pú...,A solo un día del comienzo del verano en la Ar...,https://www.laizquierdadiario.com/Economia/Mas...
1,1,\nMartes 22 de diciembre,INFORME JUNTA INTERNA ATE INDEC,Es el monto de una canasta de consumos mínimos...,Ningún trabajador debería ganar menos de $ 78...,La Junta Interna de ATE Indec estimó la canast...,https://www.laizquierdadiario.com/Economia/Nin...
2,2,\nMartes 22 de diciembre,INFORME INDEC,"Los datos corresponden al informe sobre la ""Di...",La mitad de los asalariados percibe hasta $ 30...,El 50 % de los asalariados ganaba hasta $ 30.0...,https://www.laizquierdadiario.com/Economia/La-...
3,3,\nLunes 21 de diciembre,INFORME INDEC,Al mismo tiempo los precios mayoristas del con...,Se disparó el costo de la construcción: en nov...,La inflación no se detiene y el fin de año par...,https://www.laizquierdadiario.com/Economia/Se-...
4,4,\nLunes 21 de diciembre,CRISIS ECONOMICA,IRSA Propiedades Comerciales fue la más afecta...,Acciones argentinas en Wall Street cayeron has...,Las acciones de empresas argentinas que cotiza...,https://www.laizquierdadiario.com/Economia/Acc...


In [5]:
df.flyer.values[:20]

array(['LUCRO CAPITALISTA ', 'INFORME JUNTA INTERNA ATE INDEC',
       'INFORME INDEC', 'INFORME INDEC', 'CRISIS ECONOMICA', 'TUCUMÁN',
       'CRISIS ECONOMICA', 'POLITICA', 'AJUSTE FISCAL ',
       'CONGRESO NACIONAL', 'ECONOMIA NACIONAL', '¿CUÁNDO SE COBRA? ',
       'Negocio para bonistas, ajuste para trabajadores', 'INFORME INDEC',
       nan, 'TARIFAZOS', 'INFORME INDEC', 'DEUDA EXTERNA', 'NEUQUÉN',
       'CONFERENCIA DE PRENSA'], dtype=object)

### Topics
Arbitrary selection of topics from news flyers of the portal's economic section.

In [6]:
df[df.flyer.str.contains('DEUDA|Deuda|BONO', na=False)].shape

(135, 7)

In [7]:
df[df.flyer.str.contains('BRECHA|CAMBI|RESERV', na=False)].shape

(27, 7)

In [8]:
df[df.flyer.str.contains('INFLA|SUBA|PRECIO|COSTO', na=False)].shape

(34, 7)

In [9]:
df[df.flyer.str.contains('INDEC', na=False) & df.headline.str.contains('costo|precio|infla', na=False)].shape

(14, 7)

In [10]:
df[df.flyer.str.contains('FMI|FONDO', na=False)].shape

(16, 7)

In [11]:
df[df.flyer.str.contains('SALARIO|OBRER|TRABAJA', na=False)].shape

(14, 7)

### Corpus (Body)
Clipping of the selected news body

#### Topic: Inflation

In [12]:
infla_mask = df.flyer.str.contains('INFLA|SUBA|PRECIO|COSTO', na=False)
indec_mask = df.flyer.str.contains('INDEC', na=False) & df.headline.str.contains('costo|precio|infla', na=False)

In [13]:
inflation = df[infla_mask | indec_mask]
inflation.shape

(48, 7)

Removal of dicember news to match "derecha diario" timeline

In [14]:
inflation.date

3            \nLunes 21 de diciembre
23          \nMartes 15 de diciembre
24          \nMartes 15 de diciembre
40        \nMiércoles 9 de diciembre
48            \nLunes 7 de diciembre
59           \nJueves 3 de diciembre
66         \nMartes 1ro de diciembre
84           \nLunes 23 de noviembre
86           \nLunes 23 de noviembre
89           \nLunes 23 de noviembre
94         \nViernes 20 de noviembre
100         \nJueves 19 de noviembre
106         \nMartes 17 de noviembre
125        \nViernes 13 de noviembre
128         \nJueves 12 de noviembre
211        \nMiércoles 21 de octubre
215           \nMartes 20 de octubre
240           \nMartes 13 de octubre
275     \nMiércoles 30 de septiembre
292       \nViernes 25 de septiembre
299     \nMiércoles 23 de septiembre
313        \nSábado 19 de septiembre
318        \nJueves 17 de septiembre
329         \nLunes 14 de septiembre
376           \nViernes 21 de agosto
384            \nMartes 18 de agosto
453             \nJueves 23 de julio
4

In [15]:
inflation = inflation.drop(inflation[inflation.date.str.contains('diciembre|enero')].index, axis=0)
inflation.shape

(35, 7)

#### Topic: Currency Exchange

In [16]:
exchange = df.loc[df.flyer.str.contains('BRECHA|CAMBI|RESERV', na=False)]
exchange.shape

(27, 7)

In [17]:
exchange.date

78         \nJueves 26 de noviembre
104     \nMiércoles 18 de noviembre
122       \nViernes 13 de noviembre
130        \nJueves 12 de noviembre
133     \nMiércoles 11 de noviembre
141        \nMartes 10 de noviembre
155        \nViernes 6 de noviembre
164          \nLunes 2 de noviembre
180          \nJueves 29 de octubre
191          \nMartes 27 de octubre
195           \nLunes 26 de octubre
202         \nViernes 23 de octubre
212       \nMiércoles 21 de octubre
216          \nMartes 20 de octubre
219          \nMartes 20 de octubre
221           \nLunes 19 de octubre
249          \nViernes 9 de octubre
252           \nJueves 8 de octubre
258           \nMartes 6 de octubre
268          \nViernes 2 de octubre
314      \nViernes 18 de septiembre
317       \nJueves 17 de septiembre
410            \nJueves 6 de agosto
447           \nViernes 24 de julio
650             \nJueves 14 de mayo
735            \nMartes 14 de abril
1026            \nJueves 2 de enero
Name: date, dtype: object

In [18]:
exchange = exchange.drop(exchange[exchange.date.str.contains('enero')].index, axis=0)
exchange.shape

(26, 7)

#### Prepare selection and pickle

In [19]:
inflation.insert(loc=0, column='topics', value='inflation')

In [20]:
exchange.insert(loc=0, column='topics', value='exchange')

In [21]:
data_clean = inflation.append(exchange)

In [26]:
data_clean.reset_index(drop=True, inplace=True)

In [27]:
data_clean.drop('Unnamed: 0', axis=1, inplace=True)

In [29]:
pickle.dump(data_clean, open( "data/df_clean_izq.pkl", "wb" ))

#### Corpus

In [30]:
corpus = data_clean.body.values
len(corpus)

61

### Tokenization
#### Manual

In [31]:
import re
import unidecode

import matplotlib.pyplot as plt
from wordcloud import WordCloud

"Stop words", special characters, accents and numbers are removed

In [32]:
stopwords_sp = stopwords.words('spanish')

In [33]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tockens = word_tokenize(no_accents)
    
    tockens_clean = [tocken for tocken in tockens if tocken not in stopwords_sp and tocken.isalpha()]
    
    terminos = tockens_clean

    return terminos

In [34]:
tokenized = [tokenizer(documento) for documento in corpus]

In [35]:
vocabulario = sorted(list(set([word for group in tokenized for word in group])))

In [36]:
len(vocabulario)

3244

#### CountVectorizer

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

Removal of special characters function to load in vectorizer

In [38]:
def removal(text):
    text = re.sub(r'(\d|\$|\%|\+)', '', text.lower())
    return re.sub(r'\d+', '', text)

In [39]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode', preprocessor=removal)

In [40]:
cv_tokens = vectorizer.fit_transform(corpus)

In [41]:
len(vectorizer.vocabulary_), len(vectorizer.get_feature_names())

(3034, 3034)

### Pickle

#### Manual tokens

In [42]:
def token_matrix_maker(data, vocabulario):
    matriz = np.zeros(shape = (len(data), len(vocabulario)), dtype='int')
    for i, documento in enumerate(data):
        for termino in documento:
            matriz[i, vocabulario.index(termino)] += 1
    return matriz

In [43]:
matrix = token_matrix_maker(tokenized, vocabulario)

In [44]:
# make dataframe
token_df = pd.DataFrame(matrix, columns=vocabulario, index = ['doc' + str(i + 1) for i in range(len(tokenized))] )

In [45]:
# add classification column
topics = ['inflation' for i in range(len(inflation))] + ['exchange' for i in range(len(exchange))]
token_df.insert(loc=0, column='topics', value=topics)

In [46]:
token_df.to_pickle('tokens/topic_tokens_izq.pkl')

#### CV tokens

In [47]:
pickle.dump(cv_tokens, open( "tokens/cv_izq.pkl", "wb" ))